# Comment créer une base de données vectorielle avec PostgreSLQ et pgvector

## Etapes

1. Créer la BDD PostgreSQL + pgvector dans un conteneur Docker
2. Se connecter avec psql (pour vérification)
3. Importer les données (textes)
4. Créer les vecteurs avec Sentence Transformers
5. Créer une table 'quora' dans la BDD
6. Sauvegarder les données et les vecteurs dans la BDD
7. Utiliser la recherche vectorielle 

## 1. Créer la BDD PostgreSQL + pgvector avec dans un conteneur 

### Docker-compose.yaml

Tout d'abord, nous devons créer un fichier `docker-compose.yml` avec les services nécessaires.

Dans ce fichier, nous définissons un service appelé `db` qui est basé sur l'image Docker `pgvector/pgvector:pg16`. Le service expose le port `5432` pour interagir avec la base de données et configure des variables d'environnement pour le nom de la base de données, l'utilisateur, le mot de passe et la méthode d'authentification. De plus, nous montons un fichier `init.sql` dans le répertoire `/docker-entrypoint-initdb.d` à l'intérieur du conteneur à des fins d'initialisation.

### Fichier init.sql

Dans ce script `init.sql`, nous activons l'extension `pgvector`, si elle n'existe pas déjà. Ensuite, nous créons une table appelée `embedding` avec les colonnes : `id`, `embedding`, `text` et `created_at`.

### Lancer le conteneur

```
docker-compose up -d
```

Cette commande créera un conteneur Docker avec le serveur PostgreSQL et l'extension `pgvector` déjà installés et configurés, en fonction des spécifications du fichier `docker-compose.yml`.


#### Obsolète 

Créer un base de données PostgreSQL avec `docker run`

```
docker run -d --name postgresCont -p 5432:5432 -e POSTGRES_PASSWORD=pass123 postgres
docker run -d --name postgresCont -p 5432:5432 -e POSTGRES_PASSWORD=pass123 pgvector/pgvector:pg16
docker exec -it postgresCont bash
psql -h localhost -U postgres
```

## 2. Se connecter avec psql

### Aller dans le conteneur

`docker exec -it <container id> bash`

### Se conncter à la base de donnée avec `psql`

`psql -h localhost -U postgres -d vectordb`

In [ ]:
# ! pip install sentence-transformers "psycopg[binary]" --quiet

## 3. Importer les phrases et les vecteurs

In [86]:
import os
import pickle
import time
import pandas as pd

dataset_url = "https://www.data.gouv.fr/fr/datasets/r/14159082-d1be-417e-a67c-c3c494c7a4ad"

dataset_df = pd.read_csv(dataset_url)

In [93]:
dataset_df.shape

(11812, 2)

In [149]:
corpus_questions = dataset_df.loc[:, 'question'].tolist()
corpus_reponses = dataset_df.loc[:, 'reponse'].tolist()

In [150]:
for question, reponse in zip(corpus_questions[:10], corpus_reponses[:10]):
    print(f"Question : {question} | Réponse : {reponse}")

Question : Quel architecte fût à l'origine des plans du Woolworth building? | Réponse : Cass Gilbert
Question : Où se trouvait Franck Woolworth lors de l'inauguration de son immeuble New Yorkais ? | Réponse : Washington
Question : Comment fût payé le bâtiment commandé par Franck Woolworth? | Réponse : en cash
Question : En quelle année ouvrit le Woolworth Building ? | Réponse : 1913
Question : Qui commanda la construction du Woolworth Building ? | Réponse : Frank Woolworth
Question : Quelle femme devint reine aux côtés de Philippe le Bel ? | Réponse : Jeanne Ire de Navarre
Question : Quel créancier du roi fut supprimé en 1312 ? | Réponse : l'ordre du Temple
Question : Quel créancier du roi fut supprimé en 1312 ? | Réponse : l'ordre du Temple
Question : Quelle raison pousse Philippe Le Bel à organiser les premiers Etats généraux ? | Réponse : pour lever de nouveaux impôts
Question : Quel souverain utilise les dévaluations monétaires pour s'enrichir ? | Réponse : Philippe IV le Bel


## 4. Encoder les questions avec Sentence Transformers

In [96]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

embeddings_questions = model.encode(corpus_questions, show_progress_bar=True).tolist()

Batches:   0%|          | 0/370 [00:00<?, ?it/s]

## 5. Créer une table 'piaf' dans la BDD

### Connexion à la base de données avec Psycopg

- https://www.psycopg.org/psycopg3/docs/basic/usage.html

In [97]:
import psycopg

# DEFINE THE DATABASE CREDENTIALS
user = 'testuser'
password = 'testpwd'
host = 'localhost'
port = 5432
database = 'vectordb'

db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"

### Effacer la table pour éviter d'ajouter des lignes en double

In [ ]:
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        cur.execute("""
            DROP TABLE piaf; 
            """)
        # Make the changes to the database persistent
        conn.commit()

### Création de la table

In [99]:
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS piaf (
                id serial PRIMARY KEY,
                question text NOT NULL,
                reponse text NOT NULL,
                embedding vector(384) NOT NULL,
                created_at timestamptz DEFAULT now()
                );
            """)
        # Make the changes to the database persistent
        conn.commit()

## 6. Sauvegarder les données et les vecteurs dans la BDD

In [100]:
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command
        for question, reponse, embedding in zip(corpus_questions, corpus_reponses, embeddings_questions):
            cur.execute("""INSERT INTO piaf (question, reponse, embedding) VALUES (%s, %s, %s);""", (question, reponse, embedding))
        # Make the changes to the database persistent
        conn.commit()

## 7. Utiliser la recherche vectorielle 

- https://huggingface.co/sentence-transformers/quora-distilbert-multilingual

### Rechercher par requète textuelle

In [215]:
%%time
query = "Quelle est la date de la révolution ?"

# Encoder la requète avec Sentence Transformers
embedding_query = model.encode(query, show_progress_bar=True).tolist()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 30.2 ms, sys: 0 ns, total: 30.2 ms
Wall time: 28.3 ms


In [216]:
%%time
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        res = cur.execute("""
            SELECT
                id,
                1 - (embedding <=> %s) AS cosine_similarity,
                question,
                reponse
            FROM piaf ORDER BY cosine_similarity DESC LIMIT 10;
        """, (str(embedding_query), )).fetchall()
        for row in res:
            print(f"id : {row[0]} | score : {round(row[1], 4)} | Question : {row[2]} | Réponse : {row[3]}")

id : 790 | score : 0.8086 | Question : Quelle est la conséquence de la Révolution de 1911 ? | Réponse : la proclamation de la république de Chine
id : 3835 | score : 0.7912 | Question : Quel est un des éléments de l'avènement de la révolution ? | Réponse : la baisse des salaires réels
id : 4156 | score : 0.7891 | Question : quelle est la date de la réforme? | Réponse : 1995
id : 4155 | score : 0.7891 | Question : quelle est la date de la réforme? | Réponse : 1995
id : 7495 | score : 0.7807 | Question : Quelle est la date de la libération de Paris ? | Réponse : 25 août 1944
id : 3762 | score : 0.7576 | Question : Quelle est la date de l’inauguration ? | Réponse : 11 octobre 1854
id : 3763 | score : 0.7576 | Question : Quelle est la date de l’inauguration ? | Réponse : le 11 octobre 1854
id : 7246 | score : 0.7532 | Question : De quand date la constitution ? | Réponse : 1974
id : 11371 | score : 0.7186 | Question : Quelle est la date exacte du meurtre qui déclencha la guerre ? | Réponse 

In [217]:
# Retourner les résultats dans une DataFrame
def query_to_dataframe(query, column_names):

    # Connect to an existing database
    with psycopg.connect(conninfo=db_url) as conn:
        # Open a cursor to perform database operations
        with conn.cursor() as cur:
            embeddings_query = model.encode(query).tolist()
            # Execute a command: this creates a new table
            res = cur.execute("""
                SELECT
                    id,
                    1 - (embedding <=> %s) AS cosine_similarity,
                    question,
                    reponse
                FROM piaf ORDER BY cosine_similarity DESC LIMIT 10;
            """, (str(embeddings_query), )).fetchall()
    res_df = pd.DataFrame(res, columns=column_names)
    return res_df

In [218]:
%%time
# Cherchez une question en changeant cette requète
query = "Quelle est la date de la libération ?"
query_to_dataframe(query, ['id', 'cosine_similarity', 'question', 'reponse'])

CPU times: user 30.7 ms, sys: 0 ns, total: 30.7 ms
Wall time: 47.9 ms


,id,cosine_similarity,question,reponse
0,7495,0.898055,Quelle est la date de la libération de Paris ?,25 août 1944
1,3763,0.759356,Quelle est la date de l’inauguration ?,le 11 octobre 1854
2,3762,0.759356,Quelle est la date de l’inauguration ?,11 octobre 1854
3,4155,0.755041,quelle est la date de la réforme?,1995
4,4156,0.755041,quelle est la date de la réforme?,1995
5,7810,0.748075,De quand date le début de la domination angevine?,1301
6,1019,0.746150,Quelle est la date de la fin de la tournée sec...,le 10 juin 2007
7,4256,0.738092,Quelle est la date de la reddition de la Grand...,juillet 1857
8,4257,0.738092,Quelle est la date de la reddition de la Grand...,juillet 1857
9,6857,0.726841,quelle est la date de naissance de cézanne?,19 janvier 1839


### Les plus proches voisins d'une ligne (id aléatoire)

In [219]:
import random
rand_id = random.randint(0,11812)

print(f"Random id : {rand_id}")
print()

# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        res = cur.execute("""
            SELECT
            id,
            1 - (embedding <=> (SELECT embedding FROM piaf WHERE id = %s)) AS cosine_similarity,
            question,
            reponse          
            FROM piaf
            ORDER BY cosine_similarity DESC LIMIT 20;
        """, (rand_id, )).fetchall()

        res_df = pd.DataFrame(res, columns=['id', 'cosine_similarity', 'question', 'reponse'])
res_df

Random id : 1336



,id,cosine_similarity,question,reponse
0,1334,1.000000,Quel type de vin accompagne le déjeuner ?,vin rouge produit dans la région du Balaton
1,1335,1.000000,Quel type de vin accompagne le déjeuner ?,produit dans la région du Balaton ou dans les ...
2,1336,1.000000,Quel type de vin accompagne le déjeuner ?,vin rouge
3,1330,0.768863,Où est produit le vin servi avec le déjeuner ?,dans la région du Balaton ou dans les massifs ...
4,8575,0.711023,Quel est l'enjeu du vin ?,un féroce combat où l'argent le dispute à la g...
5,8576,0.634816,Quel est le symbole du vin ?,symbole de civilisation
6,6567,0.620345,quels sont les enjeux,"enjeux locaux économiques, techniques, et surt..."
7,3513,0.614405,Combien de personnes accompagnent De Mons ?,80
8,3512,0.614405,Combien de personnes accompagnent De Mons ?,80
9,9838,0.609765,Combien rapporte Deberny à Peignot lors de leu...,"2,6 millions"
